# 7. Bandit Algorithms

## Stochastic k-armed bandits
Like in online supervised learning, the algorithm runs over time, and at every time point we need to make a decision.

However, unlike online learning, we only receive information about the action we choose, i.e. we only have one single "try".

In e.g. OCP, at any time point we get a new $x$, and we can try a plethora of different ways to update our model ($w$) (hypothetically) and see how good the new potential model is. With bandits, you don't know how good your choice is until you commit to it and do it (pull the arm), by which time, you can no longer change it.

## Hoeffding's inequality and UCB1
> [...] provides an upper bound on the probability that the sum of random variables deviates from its expected value. 
>
> -- Wikipedia

Let $X_1,\dots,X_m$ be i.i.d. random variables taking values in $[0, 1]$.

The real mean $\mu = \mathbb{E}[X]$ is unknown.

We have an empirical estimate based on our $m$ trials:

\begin{equation}
    \hat{\mu}_m = \frac{1}{m} \sum_{i = 1}^{m} X_i
\end{equation}

Then we have:

\begin{equation}
    P\left(\left|\mu - \hat{\mu}_m\right| \ge b\right) \le 2 \exp\left(-2b^2m \right) = \delta_m
\end{equation}

That is, the probability of our operation being more than $b$ off the real value is smaller than a computable threshold.

We just want an upper bound. So we set it to, say, $\delta_m$, and then compute the corresponding $b$, as a function of $\delta_m$ and $m$.

For a fixed upper probability bound, we fix $\delta_m$ and get $b = \sqrt{\frac{1}{2m} \log{\frac{2}{\delta_m}}}$.

Now we also want to set an upper bound on the probability not just for a single $m$, but for all $m$. Want upper bound for $E_m = P(|\mu - \hat{\mu}_m| \ge b)$, i.e. a lower bound for $P(|\mu - \hat{\mu}_t| \ge b, \> \forall t)$.

So we get:

\begin{equation}
\begin{aligned}
    P(|\mu - \hat{\mu}_t| \ge b, \> \forall t) & = 1 - P(E_1 \cup E_2 \cup \dots) \\
    & \ge 1 - \sum_{t=1}^{\infty} P(E_t) \\
    & \ge 1 - \sum_{t=1}^{\infty} \delta_t \\
    & \ge 1 - \delta \> \text{with} \> \delta < \sum_{t=1}^{\infty}\delta_t
\end{aligned}
\end{equation}

2nd row smaller since the sum we're subtracting is bigger (longer than the prev.).
3rd row smaller because all deltas are larger than their corresponding Es, as defined by Hoeffding's inequality itself.

We therefore want a sum of $d_t$ which is bounded. Setting $\delta_t = \frac{c}{t^2}$ works well, since the correspoinding sum converges, so the upper bound $\delta$ exists and is finite.

We now have a good heuristic: at any time step $t$, our upper bound should be $\delta_t = \frac{c}{t^2}$.

Recall that we want to express $b$ as a function of $\delta_t$, since $b$ is *the value which actually defines our upper confidence bound*.

(Note that this probability shrinks quadratically over time, so it keeps getting tighter and tighter for all arms, as we keep playing.)

All we need to do now is shove our $\delta_t$ in the formula for $b$, and we get (setting $c := 2$):

\begin{aligned}
\operatorname{UCB}(i) 
& = \hat{\mu}_i + \sqrt{\frac{1}{n_i} \ln (2  \frac{t^2}{2}) } \\
& = \hat{\mu}_i + \sqrt{\frac{\ln{t^2}}{n_i}} \\
& = \hat{\mu}_i + \sqrt{\frac{2 \ln{t}}{n_i}}
\end{aligned}

We can plug this formula right into a program! See `bonus/tutorial-bandits.ipynb` for an implementation.

TODO: there seems to be a "2" missing somewhere. Low priority.

It can be shown that UCB is a **no-regret** algorithm. ($R_T / T \rightarrow 0 \> \text{as} \> T \rightarrow \infty$)

## Applications of bandit algorithms
Non-DM:
 * Clinical trials (give the best possible cure to a patient, while also working on improving the accuracy of our diagnostics)
 * Matching markets (TODO: more info)
 * Asset pricing
 * Adaptive routing
 * Go
 
DM:
 * Advertising
 * Optimizing relevance (e.g. news article recommendations)
 * Scheduling web crawlers
 * Optimizing user interfaces (e.g. smart A/B testing)

## Contextual bandits
Also incorporate some info about every arm and every user. Useful when e.g. recommending articles, since it takes users topic preferences into account.

We still use **cummulative (contextual) regret** as a metric, $R_t = \sum_{t=1}^{T}r_t$.

Can achieve *sublinear regret* by learning **optimal mapping** from contexts (e.g. user <-> article features) to actions.

### Linear recommendations

Want to minimize regularized square loss for

\begin{equation}
    \hat{w}_i = \arg \min_w \sum_{t=1}^{m} (y_t - w^T z_t)^2 + \|w\|_2^2
\end{equation}

Note: This model only takes user features into account. And every article has its own $\hat{w}$. This is linear regression and it's easy to solve.

Key idea: Want to merge UCB and regression by having an upper confidence bound (UCB) for our $w$s. Ideally, just as in UCB1, this bound will shrink towards $w$ as time goes on.

This is LinUCB.

\begin{aligned}
 \left| \> \text{estimated reward} - \text{true reward} \> \right| & \le \text{some bound} \quad \text{(with some probability)} \\
 \left| \hat{w}^T_i z_t - w^T_i z_t \right| & 
 \le \alpha\sqrt{z^T_t(D^T_i D_i + I)^{-1}z_t}, \> p \ge 1 - \delta \\
 \left| \hat{w}^T_i z_t - w^T_i z_t \right| & 
 \le \alpha\sqrt{z^T_t M_i z_t}, \> p \ge 1 - \delta
\end{aligned}

This holds as long as $\alpha = 1 + \sqrt{\frac{1}{2} \ln \left( \frac{2}{\delta} \right)}$.

We set our desired probability bound, compute $\alpha$ and we have an algorithm!

LinUCB is also no-regret. TODO: make sure!

### Problem with linear recommendations

No shared effect modeling. We optimize every arm separately based on what users like it, but there's no way to directly exploit the fact that similar users may like similar articles.

Use hybrid models!

### Hybrid LinUCB

\begin{equation}
    y_t = w_i^T z_t + \beta^T \phi(x_i, z_t) + \epsilon_t
\end{equation}

 * $\phi(x, z)$ simply flattens (like `numpy.ravel`) $x_i z_t^T$.
 * $w_i$ is an arm's model
 * $\beta$ captures user-article similarity (i.e. user interests).
 
Can also solve this using regularized regression. We also need to compute confidence intervals for this bad boy. The algorithm is fluffy, but it works.

## Practical implementation of contextual bandits